# Grand Summary Visualization

## Detailed Analysis

This script generates the "Grand Summary" figure, a three-panel overview of the physical properties of the generated neutron star populations. It aggregates data from thousands of individual simulation curves to visualize the statistical distribution of the dataset across three domains: Thermodynamics (Equation of State), Macroscopic Structure (Mass-Radius), and Deformability (Tidal-Mass).

The script is used to compare the Hadronic and Quark populations globally, highlighting distinct behaviors in stiffness, compactness, and tidal deformability, and comparing these against known theoretical and observational constraints (such as the Buchdahl limit and GW170817).

## Physics and Math

The script visualizes relations derived from the TOV and Riccati equations.

### 1. Equation of State (Thermodynamics)
Panel A displays pressure $P$ as a function of energy density $\epsilon$. Two theoretical limits are drawn for reference:
*   **Causal Limit:** $P = \epsilon$, implying the speed of sound $c_s = 1$ (speed of light).
*   **Conformal Limit:** $P = \epsilon/3$, implying $c_s^2 = 1/3$, a limit expected in high-density deconfined QCD matter.

### 2. Mass-Radius Relation (Structure)
Panel B visualizes the solution space of the TOV equations. The **Buchdahl Limit** represents the maximum compactness allowed by General Relativity for a static sphere of fluid:

$$
R > \frac{9}{4} \frac{GM}{c^2}
$$

In the code, this is calculated using the conversion factor `A_CONV` ($\approx 1.477$ km/$M_{\odot}$):

$$
R_{Buchdahl} \approx 2.25 \times \text{A\_CONV} \times M
$$

### 3. Tidal Deformability
Panel C shows the dimensionless tidal deformability $\Lambda$ vs. Mass. The observational constraint from the binary neutron star merger **GW170817** is applied, which places an upper bound on deformability for a $1.4 M_{\odot}$ star:

$$
\Lambda_{1.4} \lesssim 580
$$

## Code Walkthrough

### 1. Configuration and Grids
Common grids are defined to facilitate the statistical aggregation of distinct curves.
*   `eps_grid`: Logarithmically spaced grid for interpolating EoS curves.
*   `mass_grid`: Linearly spaced grid for interpolating Tidal Deformability curves.

### 2. Data Processing
The script iterates through the grouped `Curve_ID`s to process the data.
*   **Interpolation:** Because each simulation produces points at different pressures and masses, `scipy.interpolate.interp1d` is used to map each curve onto the common grids.
*   **Collection:**
    *   Interpolated $P(\epsilon)$ arrays are stacked into matrices (`eos_h_stack`, `eos_q_stack`).
    *   Interpolated $\Lambda(M)$ arrays are stacked into matrices (`lam_h_stack`, `lam_q_stack`).
    *   Raw Mass and Radius points are collected for Kernel Density Estimation (KDE).

### 3. Smoothing Helper
A helper function `plot_smooth_band` is defined to visualize the statistical spread.
*   It computes the median (50th percentile) and the 90% confidence interval (5th to 95th percentiles) at each grid point.
*   Gaussian smoothing (`gaussian_filter1d`) is applied to the resulting percentile curves to reduce numerical noise from the interpolation steps.
*   The bands are plotted with transparency. Quark models are distinguished by a hatched pattern.

### 4. Panel Generation

**Panel A (EoS):**
The interpolated pressure bands are plotted. The causal and conformal limits are added as dotted lines to frame the stiffness of the models.

**Panel B (Mass-Radius):**
Instead of plotting individual lines (which would clutter the figure), `scipy.stats.gaussian_kde` is used to generate density contours.
*   This visualizes the probability density of finding a star in a specific region of the M-R plane.
*   The Buchdahl limit and the mass of PSR J0740+66 are plotted as constraints.

**Panel C (Tidal Deformability):**
The interpolated $\Lambda$ bands are plotted on a logarithmic scale. A box representing the GW170817 constraint is drawn at $M=1.4 M_{\odot}$.

## Visualization Output

The script saves a single file: `plots/fig_grand_summary.pdf`.

*   **Panel (a) Equation of State:**
    *   **X-axis:** Energy Density [MeV/fm$^3$].
    *   **Y-axis:** Pressure [MeV/fm$^3$].
    *   **Features:** Green band (Hadronic) vs. Magenta hatched band (Quark). Quark models typically show lower pressures at low densities (due to vacuum pressure $B$) but stiffen to follow the causal limit.

*   **Panel (b) Mass-Radius Relation:**
    *   **X-axis:** Radius [km].
    *   **Y-axis:** Mass [$M_{\odot}$].
    *   **Features:** Density contours showing the distinct "islands" of Hadronic and Quark stars. The "GR Forbidden" wedge marks the Buchdahl limit.

*   **Panel (c) Tidal Deformability:**
    *   **X-axis:** Mass [$M_{\odot}$].
    *   **Y-axis:** Tidal Deformability $\Lambda$ (Log scale).
    *   **Features:** Shows how $\Lambda$ decreases rapidly with mass. The GW170817 marker shows that most generated models are consistent with observational data.